In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls

kafka.yaml         test_kafka.ipynb   test_spark.ipynb
test_crawler.ipynb test_ray.ipynb


In [2]:
SRC = "../src"

In [3]:
import os
import sys
import ray

from pathlib import Path
from ray.util import inspect_serializability

sys.path.insert(0, str(Path.cwd().parents[0]))

from core.crawler import TwitterCrawler

In [14]:
import ray
from ray.util import inspect_serializability

from module.crawler.crawler import App
from base.tweet import Tweet

In [4]:
c = App(scraper_id="SEARCH", query="dataengineering", dest=None, max_results=20)

2022-06-26 17:19 | [  INFO  ] base.model.__attrs_post_init__.43  - config: {'dest': None, 'scraper_id': 'SEARCH', 'query': ['dataengineering'], 'max_results': 20}


In [6]:
output = c.run()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 710.66it/s]


In [25]:
Tweet(**output[0][0].__dict__)

TypeError: __init__() got an unexpected keyword argument 'date'

In [17]:
print(output[0][0])

https://twitter.com/dataman3180/status/1540985060182327296


In [54]:
class Test:
    @ray.remote
    def test(self, i):
        print(f"count {i}")
        
ray.init(
    num_cpus=os.cpu_count() - 2,
    ignore_reinit_error=True,
)
t = Test()
tasks = [t.test.remote() for i in range(100)]
output = ray.get(tasks)

2022-06-08 00:37:53,103	INFO worker.py:963 -- Calling ray.init() again after it has already been called.


TypeError: missing a required argument: 'self'

In [35]:
tasks = [crawler._scrape.remote(scraper) for scraper in scrapers]
output = ray.get(tasks)

TypeError: missing a required argument: 'scraper'

In [11]:
selected_tweets = [tweet.url for tweet in tweets]
crawler = App(dest=None, scraper_id="SEARCH", query=selected_tweets)
scrapers = crawler._get_scrapers()
len(scrapers)

2022-06-07 23:57 | [  INFO  ] base.model.__attrs_post_init__.44  - config: {'dest': None, 'scraper_id': 'SEARCH', 'query': ['https://twitter.com/kishorerabhinav/status/1534198493236981760', 'https://twitter.com/codexoriginal/status/1534196383330480130', 'https://twitter.com/mbnsolutions/status/1534196056963022850', 'https://twitter.com/DataSciTechScot/status/1534196056925274114', 'https://twitter.com/DataEngiLatam/status/1534185021292130305', 'https://twitter.com/TeseraSystems/status/1534184546261819392', 'https://twitter.com/corpus_news/status/1534182573450174466', 'https://twitter.com/DrMattCrowson/status/1534178251731369984', 'https://twitter.com/datapronetwork/status/1534177495695515651', 'https://twitter.com/Molecula/status/1534174843054202881', 'https://twitter.com/ZainullAbiddinD/status/1534173376163479555', 'https://twitter.com/tivonjohnson_/status/1534165926651875328', 'https://twitter.com/SalMancuso/status/1534165705439907840', 'https://twitter.com/acceldataio/status/15341610

51

In [12]:
%%time

ray.init(ignore_reinit_error=True)

@ray.remote
def scrape(scraper):
    return list(scraper.get_items())

tweets = [scrape.remote(scraper) for scraper in scrapers]
output = ray.get((tweets))

CPU times: user 135 ms, sys: 126 ms, total: 261 ms
Wall time: 13.3 s


In [15]:
len(output)

48

In [16]:
from tqdm import tqdm

In [17]:
%%time

output = []
for scraper in tqdm(scrapers):
    output.append(list(scraper.get_items()) )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [01:28<00:00,  1.73s/it]

CPU times: user 1.95 s, sys: 522 ms, total: 2.48 s
Wall time: 1min 28s


In [19]:
import os

os.cpu_count()

12